In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from bs4 import BeautifulSoup
import seaborn as sns
import re

%matplotlib notebook

In [14]:
# gets gdp data for country specified
def get_gdp_data(country):
    url = 'https://www.macrotrends.net/countries/' + country + '/gdp-per-capita'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    tables = soup.findChildren('table') # gets all tables
    my_table = tables[1] # table with US gdp per capita data
    rows = my_table.findChildren(['tr']) # separates each row of data

    year = pd.Series(map(lambda x: x.findChildren('td')[0].text,rows[2:]))
    gdp2 = pd.Series(map(lambda x: x.findChildren('td')[1].text,rows[2:]))
    annual_growth_rate = pd.Series(map(lambda x: x.findChildren('td')[2].text,rows[2:]))

    df2 = pd.DataFrame([])
    df2['Year'] = year.astype(str)
    df2['GDP string'] = gdp2
    df2['Annual Growth Rate'] = annual_growth_rate
    country_name = country.split('/')[1].replace('-', ' ')
    df2['Country'] = [country_name for i in range(0,len(df2))]
    
    # final cleaning
    df2['GDP string'] = df2['GDP string'].apply(lambda x: x.replace('(Measured)', '')).apply(lambda x: x.replace('(Forecasted)', ''))
    df2['GDP per capita'] = df2['GDP string'].apply(lambda x: x.replace(r'$', '')).apply(lambda x: x.replace(r',', '')).astype(float)
    return df2


In [15]:
# Data for top ten most populated countries
countries = ['CHN/china', 'IND/india', 'USA/united-states', 'IDN/indonesia', 'PAK/pakistan', 
             'NGA/nigeria', 'BRA/brazil', 'BGD/bangladesh', 'RUS/russia', 'MEX/mexico']

# Contains list of dataframes with each dataframe containing country data
data = pd.concat(list(map(get_gdp_data, countries)))

In [16]:
# Get Population for top ten countries
url = 'https://www.census.gov/popclock/print.php?component=counter'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
_ = re.findall(r'rows":\{(.*?)\}', soup.text)[0].split(',')

# filter out country names and populations
country_names = list(map(lambda x: x.split(':')[0].replace('"', '').lower(), _ ))
populations = list(map(lambda x: float(x.split(':')[1]), _ ))
# create a dataframe for this new data
pop_df = pd.DataFrame([])
pop_df['Country'] = country_names
pop_df['Population'] = populations

# merge data with pop_df
data = pd.merge(data, pop_df, how='inner', left_on='Country', right_on='Country')
#data = data.drop(['Population_x', 'Population_y'], axis=1)

# more cleaning
data['Annual Growth Rate'] = data['Annual Growth Rate'].apply(lambda x: float(x.replace('%', '')))
data['Total GDP'] = data['Population'] * data['GDP per capita']
data['Year'] = data['Year'].astype(int)

data.to_csv('gdp_data.csv')
data


,Year,GDP string,Annual Growth Rate,Country,GDP per capita,Population,Total GDP
0,2019,"$10,262",2.86,china,10262.0,1.394016e+09,1.430539e+13
1,2018,"$9,977",12.36,china,9977.0,1.394016e+09,1.390810e+13
2,2017,"$8,879",8.98,china,8879.0,1.394016e+09,1.237747e+13
3,2016,"$8,148",1.00,china,8148.0,1.394016e+09,1.135844e+13
4,2015,"$8,067",5.06,china,8067.0,1.394016e+09,1.124553e+13
5,2014,"$7,679",8.91,china,7679.0,1.394016e+09,1.070465e+13
6,2013,"$7,051",11.62,china,7051.0,1.394016e+09,9.829207e+12
7,2012,"$6,317",12.44,china,6317.0,1.394016e+09,8.805999e+12
8,2011,"$5,618",23.46,china,5618.0,1.394016e+09,7.831582e+12
9,2010,"$4,550",18.74,china,4550.0,1.394016e+09,6.342773e+12
